In [ ]:
import numpy as np
from scipy import integrate
from matplotlib.pylab import *
import pandas as pd
# from sklearn import preprocessing

source https://learnche.org/3E4/Assignment_6_-_2010_-_Solutions

# Model definition

In [ ]:
def dy_dt(t, y):
    """ 
    y1 = dN/dt
    y2 = dB/dt
    
    INPUTS:    
        t: time 
        y: the time varying concentrations: N and B
    """
    
    V = 1600.0   # m^3    
    
    mu_max = 5.0 # 1/day
    K = 20.0     #g/m^3
    Y_B = 0.8    # effiency
    F = 5000.0   # m^3/day
    F = 5000 + 900*sin(2*np.pi/24*t -np.pi/5) # m^3/day
    
    N_in = 150.
#     # Change these time values, depending on the question
#     if t >= 75 and t <= 85:
#         N_in = 150.0
#     else:
#         N_in = 100.0 # g/m^3
            
    N = y[0]
    B = y[1]
    y = np.zeros((2,1))
    y[0] = F/V*(N_in - N) - (1/Y_B)*(mu_max*N/(K+N)) * B
    y[1] = -B*F/V + (mu_max*N/(K+N)) * B 
    return(y)

In [ ]:
Y_B = 0.5

In [ ]:
r = integrate.ode(dy_dt).set_integrator('vode', method='bdf')

# Part 2: steady-state
ICs = [33.33333333333,  53.3333333333]
t_0 = 0.0
r.set_initial_value(ICs, t_0)
t_final = 20
dt = 0.1

# Create vectors to store the solutions 
n_steps = np.floor((t_final - t_0)/dt) + 1
time = np.zeros(int(n_steps))
N = np.zeros(int(n_steps))
B = np.zeros(int(n_steps))
N[0], B[0] = ICs
k = 1

while r.successful() and r.t < t_final:
    r.integrate(r.t + dt)
    time[k] = r.t

    N[k] = r.y[0]
    B[k] = r.y[1]
    k += 1

# Clear figure window from previous simulation
clf()
plot(time, N, 'r', label='Nutrient level')
plot(time, B, 'k', label='Biomass level')
# legend(loc='best')
grid('on')

<div class="alert alert-success">
    <b>EXERCISE</b>: how can I run the model with different values of Y_B? Make Y_B modifiable
</div>

# Model calibration

In [ ]:
# let's give Y_B different values
parset = [0.2, 0.25, 0.8, 0.9, 0.3]

In [ ]:
def biomodel():
    
    r = integrate.ode(dy_dt).set_integrator('vode', method='bdf')

    # Part 2: steady-state
    ICs = [33.33333333333,  53.3333333333]
    t_0 = 0.0
    r.set_initial_value(ICs, t_0)
    t_final = 20
    dt = 0.1


    # Create vectors to store the solutions 
    n_steps = np.floor((t_final - t_0)/dt) + 1
    time = np.zeros(int(n_steps))
    N = np.zeros(int(n_steps))
    B = np.zeros(int(n_steps))
    N[0], B[0] = ICs
    k = 1

    while r.successful() and r.t < t_final:
        r.integrate(r.t + dt)
        time[k] = r.t

        N[k] = r.y[0]
        B[k] = r.y[1]
        k += 1

    # Clear figure window from previous simulation
#     clf()
    plot(time, N, 'r', label='Nutrient level')
    plot(time, B, 'k', label='Biomass level')
    legend(loc='best')
    grid('on')
    
    return(N)

In [ ]:
for i in parset:
    print(i)

In [ ]:
pos = 1
N_mod = pd.DataFrame()

for i in parset:
    Y_B = i
    N_mod[pos] = biomodel()
    pos = pos+1

In [ ]:
N_mod

<div class="alert alert-success">
    <b>EXERCISE</b>: randomly sample Y_B
</div>

In [ ]:
np.random.rand(5)

## measured values

In [ ]:
N_meas = 30 + 10*sin(2*np.pi/24*np.arange(0, t_final+dt, dt) -np.pi/5)

In [ ]:
plt.plot(N_meas);

## compare measured and modeled values 

In [ ]:
fig, ax = plt.subplots()
ax.plot(N_meas, 'rx')
ax.plot(N_mod);

In [ ]:
N_mod[1]

In [ ]:
N_mod.shape

In [ ]:
N_meas.shape

In [ ]:
residuals = N_mod[1] - N_meas

In [ ]:
residuals.plot()

In [ ]:
np.mean([np.abs(residuals)]) # MAE mean absolute error

In [ ]:
np.mean([np.sqrt(residuals**2)]) ### RMSE (root mean sq error)

In [ ]:
np.mean([(residuals**2)]) # MSE (mean sq error) range: [0, inf] optimum: 0

## multi-parameter estimation

### define parameter space and values

In [ ]:
from SALib.sample import saltelli, latin
import seaborn as sns

In [ ]:
## here we define the domain of each parameter we want to vary
problem = {
  'num_vars': 3,
  'names': ['Y_B', 'K', 'mu_max'],
  'bounds': [[0.1, 0.99], [10, 30], [1, 7]]}

# Generate samples
param_values = latin.sample(problem, 100) #calc_second_order=False)
param_values = pd.DataFrame(param_values, columns=problem['names']) 

In [ ]:
param_values.head()

In [ ]:
sns.pairplot(param_values[problem['names']], height=1); # plotting only the sampled ones

### re-define model undefining the target parameters

In [ ]:
def dy_dt(t, y):
    """ 
    y1 = dN/dt
    y2 = dB/dt
    
    INPUTS:    
        t: time 
        y: the time varying concentrations: N and B
    """
    
    V = 1600.0   # m^3    
    
    F = 5000 + 900*sin(2*np.pi/24*t -np.pi/5) # m^3/day
    
    # Change these time values, depending on the question
    if t >= 75 and t <= 85:
        N_in = 150.0
    else:
        N_in = 100.0 # g/m^3
            
    N = y[0]
    B = y[1]
    y = np.zeros((2,1))
    y[0] = F/V*(N_in - N) - (1/Y_B)*(mu_max*N/(K+N)) * B
    y[1] = -B*F/V + (mu_max*N/(K+N)) * B 
    return(y)

In [ ]:
N_mod = pd.DataFrame()

for row in param_values.index:
    Y_B = param_values['Y_B'][row]
    K = param_values['K'][row]
    mu_max = param_values['mu_max'][row]
    
    N_mod[row] = biomodel()

In [ ]:
N_mod

### multi-parametric and multi-metric model evaluation

In [ ]:
def calc_metrics3(modelled, observed):
    """
    Calucates metrics of comparison between modelled and observed variables
    
    -------
    Prameters
    
    modelled: array of floats or time serie
    observed: array of floats or time serie
    """
    
    residuals = np.abs(modelled - observed)
    df = pd.DataFrame({'MAE': np.mean([np.abs(residuals)]), ### MAE (mean absolute error) range: [0, inf] optimum: 0
                      'RMSE': np.mean([np.sqrt(residuals**2)]), ### RMSE (root mean sq error)
                      'MSE': np.mean([(residuals**2)]), ### MSE (mean sq error) range: [0, inf] optimum: 0
#                       'RRMSE': np.mean([np.sqrt((residuals**2))])/np.mean(observed), ### RRMSE (relative root mean sq error)
#                       'SSE': np.sum(residuals**2), ### SSE (sum of sq errors) range: [0, inf] optimum: 0
#                       'AMRE': np.mean([np.abs(residuals/observed)]), ### AMRE (abs mean relative error [from MREin pystran]) range: [-inf, inf] optimum: 0
#                       'MARE': np.mean([(np.abs(residuals)/observed)]), ### MARE (mean abs relative error) range: [0, inf] optimum: 0
#                       'SARE': (np.abs(residuals)/observed).sum(axis=0), ### SARE (sum of abs relative error) range: [0, inf] optimum: 0
#                       'MeAPE': np.median([(np.abs(residuals)*100./observed)]), ### MeAPE (median of absolute prediction error) range: [0, inf] optimum: 0
#                       'MSRE': np.mean([((residuals/observed)**2)]), ### MSRE (mean sq relative error) range: [0, inf] optimum: 0
                      'RVE': np.sum([residuals])/np.sum([observed])}, ### RVE (relative vol error) range: [-inf, inf] optimum: 0
                      index=[0])
    return df

In [ ]:
calc_metrics3(N_mod[0], N_meas)

<div class="alert alert-success">
    <b>EXERCISE</b>: run the cell below, why doesn't work?! fix it
</div>

In [ ]:
# scores_martix = pd.DataFrame()

for scenario in N_mod:
    a = calc_metrics3(N_mod[scenario], N_meas)
    a.index = [scenario]
    scores_martix = pd.concat([scores_martix, a], axis=0, ignore_index=False)

In [ ]:
scores_martix

## put order into all these metrics' numbers

In [ ]:
from sklearn import preprocessing

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = pd.DataFrame(min_max_scaler.fit_transform(scores_martix), columns=scores_martix.columns)
x_scaled['sum'] = x_scaled.sum(axis=1)
x_scaled['scenario'] = scores_martix.index
x_scaled = x_scaled.set_index(x_scaled['sum'], drop=False).sort_index()

In [ ]:
x_scaled

### visualization!

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 10))
sns.heatmap(x_scaled.drop(['scenario', 'sum'], axis=1)[:100], # plotting only the 12 metrics and the best 50 scenarios
            ax=ax1, annot=False, cmap="YlGnBu");

### the top 10 scenarios

In [ ]:
x_scaled['scenario'].head(10)

In [ ]:
param_values.iloc[x_scaled['scenario'].head(10)]

<div class="alert alert-success">
    <b>EXERCISE</b>: Plot the best modeled results with the measured data
</div>

In [ ]:
fig, ax = plt.subplots()
ax.plot(N_mod[x_scaled['scenario'].head(3)])
ax.plot(N_meas, 'x')

### How do my parameters distribute?

In [ ]:
threshold = 10
good = param_values[problem['names']].loc[x_scaled['scenario'].head(threshold).values[:]]
bad = param_values[problem['names']].loc[x_scaled['scenario'].tail(threshold*2).values[:]]
good['color'] = 'good'
bad['color'] = 'bad'
good_bad_nh = pd.concat([good, bad])

In [ ]:
sns.pairplot(good_bad_nh, hue='color');